In [ ]:
import jax.numpy as jnp
from jaxtyping import Float, Array

from compressible_1d import solver

In [ ]:
n_cells = 10
n_ghost_cells = 1  # per side
delta_t = 1e-4  # normalized
delta_x = 1 / n_cells  # normalized cell length
gamma = 1.4
# TODO: how do i get this
U_field: Float[Array, "3 n_cells + 2*n_ghost_cells"] = jnp.zeros(
    (3, n_cells + 2 * n_ghost_cells)
)

# one update step
U_next_field: Float[Array, "3 n_cells + 2*n_ghost_cells"] = jnp.zeros_like(U_field)

for i in range(n_ghost_cells, n_cells + n_ghost_cells):
    flux_plus = solver.lax_friedrichs(
        U_field[:, i], U_field[:, i + 1], gamma=gamma, diffusivity_scale=0.0
    )
    flux_minus = solver.lax_friedrichs(
        U_field[:, i - 1], U_field[:, i], gamma=gamma, diffusivity_scale=0.0
    )

    dU_dt = -1 / delta_x * (flux_plus - flux_minus)

    U_next_field[:, i] = U_field[:, i] + delta_t * dU_dt

    # diagnose solution